In [5]:
%pip install opencv-python tensorflow numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load FER2013 dataset
data = pd.read_csv('fer2013.csv')

# Extract pixels and emotions
pixels = data['pixels'].tolist()
emotions = data['emotion'].tolist()

# Convert pixels to numpy array
faces = []
for pixel_sequence in pixels:
    face = np.array(pixel_sequence.split(' '), dtype='float32').reshape(48, 48)
    faces.append(face)

faces = np.expand_dims(faces, -1)  # Add channel dimension
emotions = to_categorical(emotions)  # One-hot encode emotions

# Normalize pixel values to [0, 1]
faces = faces / 255.0

# Split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.2, random_state=42)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotions
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# Save the model
model.save('emotion_recognition_model.h5')

c:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 73s 159ms/step - accuracy: 0.2550 - loss: 1.8045 - val_accuracy: 0.4039 - val_loss: 1.5609
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 73s 162ms/step - accuracy: 0.4125 - loss: 1.5242 - val_accuracy: 0.4794 - val_loss: 1.3751
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 73s 163ms/step - accuracy: 0.4712 - loss: 1.3882 - val_accuracy: 0.5141 - val_loss: 1.2982
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 73s 162ms/step - accuracy: 0.4982 - loss: 1.3226 - val_accuracy: 0.5209 - val_loss: 1.2541
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 75s 166ms/step - accuracy: 0.5259 - loss: 1.2488 - val_accuracy: 0.5347 - val_loss: 1.2514
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 74s 164ms/step - accuracy: 0.5522 - loss: 1.1873 - val_accuracy: 0.5435 - val_loss: 1.2055
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 190s 423ms/step - accuracy: 0.5639 - loss: 1.1431 - val_accuracy: 0.5454 - val_loss: 1.1960
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 144s 320ms/step - accuracy: 0.5818 - loss:

In [1]:
import cv2

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
%pip install mtcnn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
from mtcnn import MTCNN
from tensorflow.keras.models import load_model

# Load the pre-trained emotion recognition model
emotion_model = load_model('emotion_recognition_model.h5')  # Replace with your model path

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Initialize MTCNN detector
detector = MTCNN()

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert frame to RGB (MTCNN expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces using MTCNN
    faces = detector.detect_faces(rgb_frame)

    for face in faces:
        x, y, width, height = face['box']
        x, y = max(0, x), max(0, y)  # Ensure coordinates are within bounds

        # Extract the face ROI
        face_roi = rgb_frame[y:y+height, x:x+width]

        try:
            # Convert to grayscale (if the model expects grayscale input)
            gray_face = cv2.cvtColor(face_roi, cv2.COLOR_RGB2GRAY)
            resized_face = cv2.resize(gray_face, (48, 48))
            normalized_face = resized_face / 255.0
            reshaped_face = np.reshape(normalized_face, (1, 48, 48, 1))  # Adjust for grayscale input

            # Predict emotion
            emotion_prediction = emotion_model.predict(reshaped_face)
            emotion_index = np.argmax(emotion_prediction)
            emotion = emotion_labels[emotion_index]

        except Exception as e:
            print(f"Prediction Error: {e}")
            emotion = "Unknown"

        # Draw rectangle around the face and display emotion
        cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with real-time results
    cv2.imshow('Real-Time Face Emotion Recognition', frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap.release()

# Close OpenCV windows properly
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━